In [1]:
!pip install torch_geometric

In [2]:
import numpy as np
import torch
from torch_geometric.data import Data
import gymnasium as gym
from gymnasium import spaces
from helpers_functions import *
from gerry_environment import FrankenmanderingEnv
from graph_initiator import build_init_data,build_inchworm_soc_init_data
import matplotlib.pyplot as plt
import pandas as pd
from random import sample,randint
# from make_grid import Graph

In [3]:
def labels_to_action(labels, num_districts, dtype=np.float32):
    """
    Convert an integer label vector (shape [N]) to an action matrix
    expected by env.step: shape (N, num_districts), each row is
    a 1-hot encoding of the desired district for that voter.
    """
    N = len(labels)

    A = torch.zeros(N, num_districts)

    for i, lab in enumerate(labels):
        if lab >= 0 and lab < num_districts:
            A[i, int(lab)] = 1.0
        else:
            # keep row zeros -> will become -1 label in env.step (avoid if possible)
            pass
    return A

In [4]:
def natural_OD(env, districts, num_districts, drf,geo_edge, Beta1,Beta2):
    assignment = labels_to_action(districts, num_districts)
    obs, reward, terminated, truncated, info = env.step(assignment,drf)
    opinions = np.array(obs.get_x())
    target = np.array(env.get_target())

    dist_target = np.linalg.norm(opinions - target, axis=1).sum()

    return obs, reward, dist_target

In [11]:
geographical_edge = torch.tensor([[0,1,2,3,3,3,3,3,3,3,4,4,4,4,5,5,6,6,7,7,7,8,8,8,8,9,9,9],
                     [3,3,3,0,1,2,4,7,8,9,3,5,8,9,4,6,5,7,3,8,6,3,4,9,7,3,4,8]])

In [28]:
history = []
initi_ops=[]

init_data, G_inch = build_inchworm_soc_init_data()
init_data.set_geo_edge(geographical_edge)
num_voters = init_data.get_x().shape[0]
num_districts = 4

dist_label = torch.tensor((1,1,2,3,4,1))
# dist_label = torch.arange(num_voters)
init_data.set_y(dist_label)

env = FrankenmanderingEnv(num_voters=num_voters,
                          num_districts=num_districts,
                          opinion_dim=1,
                          init_FrankenData=init_data,
                          target_opinion=[7,7,7,7,7,7])


obs, _ = env.reset()
 # do steps
for _ in range(50):
      obs, reward, dist_target = natural_OD(env, init_data.get_y(), num_districts, drf_inc_noso, init_data.get_geo_edge(), 0.1,0.5)
      # store data
      history.append({
                'graph_id':0,
                'opnion_id': 0,
                'obs': obs,
                'reward':reward,
                'dis_to_target': dist_target,
                'opinion': obs.get_x(),
                'init_ops':init_data.get_x(),
                'init_data': init_data
                })


In [13]:
init_data.edge_index, init_data.get_geo_edge()

(tensor([[0, 1, 1, 2, 2, 3, 3, 4, 4, 5],
         [1, 0, 2, 1, 3, 2, 4, 3, 5, 4]]),
 tensor([[0, 1, 2, 3, 3, 3, 3, 3, 3, 3, 4, 4, 4, 4, 5, 5, 6, 6, 7, 7, 7, 8, 8, 8,
          8, 9, 9, 9],
         [3, 3, 3, 0, 1, 2, 4, 7, 8, 9, 3, 5, 8, 9, 4, 6, 5, 7, 3, 8, 6, 3, 4, 9,
          7, 3, 4, 8]]))

In [29]:
df_history = pd.DataFrame(history)
df_history.to_pickle('no_normalization_drf_nosco.pkl')
len(history)

50